In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.nn import Module, Linear, Sigmoid, ReLU
from torch.autograd import Variable
import datetime

In [13]:
X_arr=[]
Y_arr=[]
#это пример с рандомными значениями
# Предположим нам надо предсказывать плотность загрузки по средним размерам коробки в контейнере
folder_path = "ALGORITM"
directory_contents = os.listdir(folder_path)
for filename in directory_contents:
    df = pd.pandas.read_json("ALGORITM//"+filename)
    data = pd.DataFrame(df['data'])['data']
    cargoes = pd.DataFrame(list(data[0]))
    sizes = pd.DataFrame(list(cargoes['size']))
    sizes = sizes.iloc[:].values
    avg_width=0
    avg_height=0
    avg_length=0
    i=0
    for size in sizes:
        avg_width += size[0]
        avg_height += size[1]
        avg_length += size[2]
        i+=1
    avg_width /= i
    avg_height /= i
    avg_length /= i
    avg_all = (avg_width+avg_height+avg_length)/i
    avg_all = 1/avg_all
    X_arr.append([avg_width,avg_height,avg_length])
    data_result = pd.DataFrame(df['data_result'])['data_result']
    cargo_space = data_result['cargo_space']
    calculation_info = cargo_space['calculation_info']['density_percent']
    Y_arr.append(calculation_info)
X_arr= np.vstack(X_arr)
X_arr
print(Y_arr)
print(X_arr)

[81.76416, 82.5, 94.44444, 74.02778, 100.0, 92.20825, 55.72132, 82.7095, 58.69138, 55.72132, 100.0, 95.3125, 99.43182, 99.43182, 80.63824, 63.38312, 62.54826, 47.6821, 47.6821, 49.93126, 47.80008, 49.37232, 55.5809, 89.15496, 76.15648, 76.15648, 99.43182, 87.27273, 90.51218, 90.51218, 100.0, 82.084, 76.33322, 79.12946, 86.95652, 68.84688, 45.11657, 94.64265, 94.64265, 79.10819, 63.64827, 98.24176, 48.6333, 48.51001, 48.51001, 48.51001, 76.12721, 88.59972, 67.50455, 79.12094, 79.12094, 79.64019, 55.07126, 83.33333, 62.0, 100.0, 80.2589, 79.86997, 72.82762, 92.54325, 92.54325, 92.54325, 91.18541, 94.56265, 95.88089, 46.64721, 58.3393, 57.48137, 96.64382, 81.76416, 78.19096, 69.94074, 73.19202, 72.06703, 72.03363, 85.92912, 76.33322, 72.06703, 55.27211, 55.27211, 55.27211, 57.39796, 42.80376, 43.66653, 69.71802, 74.09029, 71.67931, 71.67931, 69.51437, 58.35529, 84.5985, 79.69262, 95.92491, 73.18317, 98.61134, 90.51218, 100.0, 100.0, 82.78877, 69.51437, 78.05612, 80.2589, 78.05612, 85.9009

In [14]:
# В библиотеке torch для обучения данные должны быть в формате тензоров
# соответственно разделяем
# затем приводим к формату
X = torch.autograd.Variable(torch.FloatTensor(X_arr))
Y = torch.autograd.Variable(torch.FloatTensor(Y_arr))
# количество признаков потом указывает на вход в нейроне
num_features = X.shape[1]

In [15]:
# Создаем сеть то есть Sequential - последовательность
# сюда внутрь можно добавлять разные слои
# документация - https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
model = torch.nn.Sequential(
    Linear(num_features, 1),
    ReLU()
)

# Пока здесь один слой линейный - Linear
# ReLU() - функция активация, после слоя
# функция активация тоже могут быть разные - можно погулить

In [27]:
# задаем какую-нибудь функцию потерь для обучения
# здесь стандартная как для регрессии - квадратичная ошибка
loss_fn = torch.nn.MSELoss(size_average=False)

# шаг градиентного спуска (точнее -- метода оптимизации)
learning_rate = 0.0000001  
# сам метод оптимизации нейросети (обычно лучше всего по-умолчанию работает Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

for t in range(50000):
    y_pred = model(X)

    # выведем loss - смотрим - ошибка на каждом этапе уменьшается
    loss = loss_fn(y_pred, Y)
    print('{} {}'.format(t, loss.data))

    # обнуляем градиенты перед backard_pass'ом (обязательно!)
    optimizer.zero_grad()

    # backward_pass() -- вычисляем градиенты loss'а по параметрам (весам) нейросети
    # этой командой мы только вычисляем градиенты, но ещё не обновляем веса
    loss.backward()

    # а тут уже обновляем веса
    optimizer.step()

C:\Users\AliSa\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\AliSa\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([628])) that is different to the input size (torch.Size([628, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 131109584.0
1 131109584.0
2 131109576.0
3 131109576.0
4 131109584.0
5 131109584.0
6 131109584.0
7 131109576.0
8 131109576.0
9 131109584.0
10 131109576.0
11 131109576.0
12 131109584.0
13 131109584.0
14 131109576.0
15 131109584.0
16 131109584.0
17 131109584.0
18 131109576.0
19 131109576.0
20 131109576.0
21 131109576.0
22 131109576.0
23 131109584.0
24 131109584.0
25 131109584.0
26 131109568.0
27 131109584.0
28 131109584.0
29 131109576.0
30 131109576.0
31 131109576.0
32 131109576.0
33 131109576.0
34 131109584.0
35 131109584.0
36 131109584.0
37 131109584.0
38 131109576.0
39 131109576.0
40 131109576.0
41 131109576.0
42 131109576.0
43 131109584.0
44 131109584.0
45 131109584.0
46 131109584.0
47 131109584.0
48 131109584.0
49 131109584.0
50 131109576.0
51 131109576.0
52 131109576.0
53 131109576.0
54 131109584.0
55 131109568.0
56 131109576.0
57 131109584.0
58 131109576.0
59 131109584.0
60 131109576.0
61 131109576.0
62 131109584.0
63 131109584.0
64 131109584.0
65 131109584.0
66 131109584.0
67 13

552 131109584.0
553 131109584.0
554 131109584.0
555 131109584.0
556 131109584.0
557 131109576.0
558 131109584.0
559 131109584.0
560 131109584.0
561 131109576.0
562 131109584.0
563 131109584.0
564 131109584.0
565 131109584.0
566 131109584.0
567 131109576.0
568 131109576.0
569 131109576.0
570 131109584.0
571 131109584.0
572 131109584.0
573 131109584.0
574 131109584.0
575 131109584.0
576 131109584.0
577 131109576.0
578 131109576.0
579 131109584.0
580 131109584.0
581 131109584.0
582 131109584.0
583 131109584.0
584 131109584.0
585 131109584.0
586 131109576.0
587 131109576.0
588 131109576.0
589 131109584.0
590 131109584.0
591 131109584.0
592 131109584.0
593 131109584.0
594 131109584.0
595 131109584.0
596 131109576.0
597 131109576.0
598 131109584.0
599 131109584.0
600 131109584.0
601 131109584.0
602 131109584.0
603 131109584.0
604 131109584.0
605 131109584.0
606 131109584.0
607 131109584.0
608 131109584.0
609 131109584.0
610 131109584.0
611 131109584.0
612 131109576.0
613 131109576.0
614 1311

1077 131109568.0
1078 131109584.0
1079 131109576.0
1080 131109568.0
1081 131109576.0
1082 131109576.0
1083 131109576.0
1084 131109576.0
1085 131109576.0
1086 131109584.0
1087 131109584.0
1088 131109576.0
1089 131109576.0
1090 131109576.0
1091 131109584.0
1092 131109576.0
1093 131109568.0
1094 131109576.0
1095 131109584.0
1096 131109576.0
1097 131109584.0
1098 131109576.0
1099 131109576.0
1100 131109584.0
1101 131109584.0
1102 131109576.0
1103 131109584.0
1104 131109584.0
1105 131109584.0
1106 131109584.0
1107 131109576.0
1108 131109584.0
1109 131109576.0
1110 131109576.0
1111 131109584.0
1112 131109576.0
1113 131109584.0
1114 131109576.0
1115 131109584.0
1116 131109576.0
1117 131109568.0
1118 131109576.0
1119 131109576.0
1120 131109584.0
1121 131109584.0
1122 131109576.0
1123 131109576.0
1124 131109576.0
1125 131109576.0
1126 131109576.0
1127 131109576.0
1128 131109576.0
1129 131109576.0
1130 131109584.0
1131 131109576.0
1132 131109584.0
1133 131109584.0
1134 131109568.0
1135 131109584

1559 131109576.0
1560 131109584.0
1561 131109576.0
1562 131109584.0
1563 131109576.0
1564 131109576.0
1565 131109584.0
1566 131109576.0
1567 131109576.0
1568 131109584.0
1569 131109576.0
1570 131109576.0
1571 131109584.0
1572 131109576.0
1573 131109584.0
1574 131109584.0
1575 131109576.0
1576 131109576.0
1577 131109584.0
1578 131109584.0
1579 131109584.0
1580 131109576.0
1581 131109584.0
1582 131109576.0
1583 131109576.0
1584 131109584.0
1585 131109576.0
1586 131109576.0
1587 131109576.0
1588 131109584.0
1589 131109584.0
1590 131109584.0
1591 131109576.0
1592 131109576.0
1593 131109576.0
1594 131109584.0
1595 131109584.0
1596 131109576.0
1597 131109584.0
1598 131109568.0
1599 131109584.0
1600 131109584.0
1601 131109584.0
1602 131109576.0
1603 131109584.0
1604 131109584.0
1605 131109576.0
1606 131109576.0
1607 131109584.0
1608 131109584.0
1609 131109584.0
1610 131109576.0
1611 131109584.0
1612 131109584.0
1613 131109584.0
1614 131109576.0
1615 131109584.0
1616 131109576.0
1617 131109584

...

48008 131109576.0
48009 131109584.0
48010 131109576.0
48011 131109568.0
48012 131109584.0
48013 131109584.0
48014 131109568.0
48015 131109584.0
48016 131109584.0
48017 131109576.0
48018 131109576.0
48019 131109584.0
48020 131109584.0
48021 131109584.0
48022 131109584.0
48023 131109584.0
48024 131109576.0
48025 131109576.0
48026 131109584.0
48027 131109584.0
48028 131109584.0
48029 131109576.0
48030 131109576.0
48031 131109584.0
48032 131109584.0
48033 131109584.0
48034 131109576.0
48035 131109576.0
48036 131109584.0
48037 131109584.0
48038 131109584.0
48039 131109576.0
48040 131109576.0
48041 131109576.0
48042 131109584.0
48043 131109576.0
48044 131109584.0
48045 131109576.0
48046 131109576.0
48047 131109576.0
48048 131109576.0
48049 131109576.0
48050 131109584.0
48051 131109568.0
48052 131109576.0
48053 131109568.0
48054 131109584.0
48055 131109568.0
48056 131109584.0
48057 131109584.0
48058 131109584.0
48059 131109584.0
48060 131109576.0
48061 131109584.0
48062 131109584.0
48063 1311

48465 131109584.0
48466 131109576.0
48467 131109576.0
48468 131109576.0
48469 131109584.0
48470 131109584.0
48471 131109584.0
48472 131109584.0
48473 131109576.0
48474 131109584.0
48475 131109584.0
48476 131109576.0
48477 131109584.0
48478 131109576.0
48479 131109584.0
48480 131109576.0
48481 131109584.0
48482 131109576.0
48483 131109576.0
48484 131109584.0
48485 131109576.0
48486 131109576.0
48487 131109584.0
48488 131109576.0
48489 131109584.0
48490 131109576.0
48491 131109576.0
48492 131109584.0
48493 131109584.0
48494 131109584.0
48495 131109576.0
48496 131109584.0
48497 131109576.0
48498 131109576.0
48499 131109584.0
48500 131109576.0
48501 131109584.0
48502 131109584.0
48503 131109568.0
48504 131109584.0
48505 131109576.0
48506 131109576.0
48507 131109576.0
48508 131109576.0
48509 131109584.0
48510 131109584.0
48511 131109576.0
48512 131109584.0
48513 131109584.0
48514 131109576.0
48515 131109584.0
48516 131109576.0
48517 131109584.0
48518 131109576.0
48519 131109576.0
48520 1311

48944 131109584.0
48945 131109584.0
48946 131109584.0
48947 131109576.0
48948 131109584.0
48949 131109576.0
48950 131109584.0
48951 131109576.0
48952 131109584.0
48953 131109584.0
48954 131109584.0
48955 131109576.0
48956 131109584.0
48957 131109576.0
48958 131109576.0
48959 131109576.0
48960 131109576.0
48961 131109584.0
48962 131109576.0
48963 131109576.0
48964 131109584.0
48965 131109576.0
48966 131109584.0
48967 131109576.0
48968 131109576.0
48969 131109584.0
48970 131109576.0
48971 131109584.0
48972 131109576.0
48973 131109576.0
48974 131109584.0
48975 131109576.0
48976 131109584.0
48977 131109576.0
48978 131109576.0
48979 131109584.0
48980 131109576.0
48981 131109568.0
48982 131109584.0
48983 131109584.0
48984 131109584.0
48985 131109576.0
48986 131109584.0
48987 131109576.0
48988 131109584.0
48989 131109576.0
48990 131109576.0
48991 131109584.0
48992 131109576.0
48993 131109576.0
48994 131109576.0
48995 131109568.0
48996 131109584.0
48997 131109584.0
48998 131109584.0
48999 1311

49411 131109584.0
49412 131109584.0
49413 131109584.0
49414 131109576.0
49415 131109584.0
49416 131109584.0
49417 131109584.0
49418 131109576.0
49419 131109576.0
49420 131109584.0
49421 131109584.0
49422 131109584.0
49423 131109576.0
49424 131109576.0
49425 131109584.0
49426 131109576.0
49427 131109576.0
49428 131109576.0
49429 131109584.0
49430 131109584.0
49431 131109576.0
49432 131109576.0
49433 131109584.0
49434 131109584.0
49435 131109584.0
49436 131109576.0
49437 131109584.0
49438 131109584.0
49439 131109576.0
49440 131109584.0
49441 131109576.0
49442 131109568.0
49443 131109576.0
49444 131109584.0
49445 131109576.0
49446 131109576.0
49447 131109576.0
49448 131109584.0
49449 131109576.0
49450 131109576.0
49451 131109576.0
49452 131109584.0
49453 131109584.0
49454 131109584.0
49455 131109576.0
49456 131109584.0
49457 131109576.0
49458 131109584.0
49459 131109576.0
49460 131109576.0
49461 131109584.0
49462 131109576.0
49463 131109584.0
49464 131109576.0
49465 131109568.0
49466 1311

49887 131109584.0
49888 131109584.0
49889 131109584.0
49890 131109576.0
49891 131109584.0
49892 131109584.0
49893 131109568.0
49894 131109576.0
49895 131109584.0
49896 131109576.0
49897 131109576.0
49898 131109584.0
49899 131109584.0
49900 131109584.0
49901 131109584.0
49902 131109584.0
49903 131109584.0
49904 131109576.0
49905 131109584.0
49906 131109576.0
49907 131109584.0
49908 131109584.0
49909 131109584.0
49910 131109576.0
49911 131109584.0
49912 131109584.0
49913 131109576.0
49914 131109576.0
49915 131109576.0
49916 131109576.0
49917 131109584.0
49918 131109584.0
49919 131109584.0
49920 131109576.0
49921 131109576.0
49922 131109584.0
49923 131109584.0
49924 131109584.0
49925 131109576.0
49926 131109584.0
49927 131109584.0
49928 131109584.0
49929 131109584.0
49930 131109584.0
49931 131109576.0
49932 131109584.0
49933 131109576.0
49934 131109584.0
49935 131109576.0
49936 131109576.0
49937 131109584.0
49938 131109576.0
49939 131109568.0
49940 131109576.0
49941 131109576.0
49942 1311

In [26]:
df = pd.pandas.read_json("input_example.json")
data = pd.DataFrame(df['input_data'])['input_data']
groups = pd.DataFrame(list(data['groups']))
cargoes = pd.DataFrame(list(groups['cargoes']))
cargoes
avg_width=0
avg_height=0
avg_length=0
i = 0
groups = groups['cargoes']
for cargo in groups:
    avg_width += cargo[0]['width'] * cargo[0]['count']
    avg_height += cargo[0]['height'] * cargo[0]['count']
    avg_length += cargo[0]['length'] * cargo[0]['count']
    i += cargo[0]['count']
avg_width /= i
avg_height /= i
avg_length /= i

x = [30000, 250, 400]
x = torch.autograd.Variable(torch.FloatTensor(x))
c = model(x)
print(c)

tensor([76.2979], grad_fn=<ReluBackward0>)
